# PauliString (WIP)

This class provides a symbolic representation of a Pauli operator of one or many qubits.

The general form of any PauliString is 

$$c\bigotimes_{i}\sigma_i$$

where $c \in \mathbb{C}$ and $\sigma_i$ corresponds to a Pauli operator or the identity operator. We will see that, in code, the presence of identity operators is inferred.

The internal representation is a dictionary of indices with their respective operators and a scalar coefficient. N.B. indices are one-based.

In [32]:
from magpy import PauliString as PS


---
### Instantiation

The constructor takes integers and/or sets of integers corresponding to the position(s) in which to insert each operator. Any gaps are inferred to be identity operators. 


We will see later that a much tidier way to write constructors can be achieved using composition.

For example,

$$A = \sigma_x \otimes \text{Id} \otimes \sigma_x \otimes \sigma_y,$$

may be written as

In [33]:
A = PS(x={1, 3}, y=4)
A

1*{1: x, 3: x, 4: y}

MagPy also provides explicit methods for the three pauli operators. These may be used when the operator includes only one of the Pauli operators (and identity).

These constructors default to a single qubit when provided with no arguments.

For example,

$$
\begin{aligned}
B &= \text{Id} \otimes \sigma_x \otimes \sigma_x \\
C &= \sigma_y
\end{aligned}
$$

may be written as

In [34]:
B = PS.X(2, 3)
C = PS.Y()

Albeit trivial, for completeness there is an identity operator constructor:

In [35]:
Id = PS.Id()
Id

1*{}


---

### Scalar coefficients

PauliStrings have constant scalar coefficients and can perform scalar multiplication.

For example,

$$ D = 2i \, (\sigma_x \otimes \sigma_y)$$

may be written as

In [36]:
D = 2j * PS(x=1, y=2)
D

2j*{1: x, 2: y}


---

### Composition

PauliStrings may be composed. Where possible, operators will be simplified algebraically. 

The composition operator allows for writing constructors in a more mathematical manner than that which was detailed above.

The operator 

$$\sigma_x \otimes \sigma_y$$

may be written as

In [37]:
PS.X(1) * PS.Y(2)

1*{1: x, 2: y}

Note that this is different from

In [38]:
PS.X() * PS.Y()

1j*{1: z}

which is 

$$\sigma_x \sigma_y = i \, \sigma_z.$$

The first operator can be written as

$$(\sigma_x \otimes \text{Id})\,(\text{Id} \otimes \sigma_y),$$

which is the form taken in code. This layout highlights the fact that $\sigma_x$ is acting on the first qubit and $\sigma_y$ on the second.

The asterisk binary operator only performs operator composition, _not_ the tensor product. The tensor product is implied by the structure of the PauliStrings.

Here is a more sophisticated example:

$$
\begin{aligned}
E &= \sigma_y \otimes \sigma_y \otimes \text{Id} \\
F &= \sigma_x \otimes \text{Id} \otimes \sigma_z \\
\end{aligned}$$

$$EF = -i \, (\sigma_z \otimes \sigma_y \otimes \sigma_z)$$


In [39]:
E = PS.Y(1) * PS.Y(2)
F = PS.X(1) * PS.Z(3)
E * F

-1j*{1: z, 2: y, 3: z}

Here we can see the implicit appearance of the identity operator. Particularly in `E`, we can see how any number of identity operators after the last Pauli operator may be assumed to be there.

This means that our defintion of `E` is functionally equivalent in code to 

$$
\begin{aligned}
&\sigma_y \otimes \sigma_y, \\
&\sigma_y \otimes \sigma_y \otimes \text{Id}, \\
&\sigma_y \otimes \sigma_y \otimes \text{Id} \otimes \text{Id}, \\
\end{aligned}
$$

and so on. Identity operators may be explicitly included for readability's sake, although it is functionally pointless.

---

### Addition

Summing two PauliStrings results in a constant HamiltonianOperator.

The operator

$$2 \, \sigma_x + \sigma_y$$

can be written as

In [40]:
2*PS.X() + PS.Y()

{1: [2*{1: x}, 1*{1: y}]}

See [here]() for more details on HamiltonianOperators.


### Equality

Two PauliStrings are said to be equal if they contain the same qubits in the same indices and have the same scalar coefficients.

In [41]:
-PS.X(1) * PS.Y(2) == PS(x=1, y=2) * -1

True